In [115]:
import gym
env = gym.make('CartPole-v1')

In [116]:
import numpy as np
def random_policy(obs):
    return np.random.randint(env.action_space.n)

In [117]:
import time
from copy import deepcopy
#Visualise agent function
def visualise_agent(policy, command, n=5):
    try:
        for trial_i in range(n):
            current_command = deepcopy(command)
            observation = env.reset()
            done=False
            t=0
            episode_return=0
            while not done:
                env.render()
                network_input = torch.tensor(np.append(observation, current_command)).double()
                policy_action = policy(network_input)
                observation, reward, done, info = env.step(policy_action)
                episode_return+=reward
                #time.sleep(0.1)
                current_command[0]-=reward
                current_command[1] = max(1, current_command[1]-1)
                
                t+=1
            env.render()
            time.sleep(1.5)
            print("Episode {} finished after {} timesteps. Return = {}".format(trial_i, t, episode_return))
        env.close()
    except KeyboardInterrupt:
        env.close()

In [118]:
#visualise_agent(random_policy, command=[500, 500], 1)

In [174]:
import torch
import torch.nn.functional as F

class FCNN_AGENT(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = torch.nn.Sequential(
            torch.nn.Linear(np.prod(env.observation_space.shape)+2, 10),
            torch.nn.ReLU(),
            torch.nn.Linear(10, 10),
            torch.nn.ReLU(),
            torch.nn.Linear(10, env.action_space.n)
        )
    
    def forward(self, x):
        x = self.fc(x)
        return F.softmax(x, dim=-1)
    
    def create_optimizer(self, lr):
        self.optimizer = torch.optim.Adam(self.parameters(), lr=lr)

In [175]:
from copy import deepcopy
def collect_experience(policy, replay_buffer, replay_size, last_few, n_episodes=100):
    global i_episode
    init_replay_buffer = deepcopy(replay_buffer)
    try:
        for _ in range(n_episodes):
            command = sample_command(init_replay_buffer, last_few)
            writer.add_scalar('Command desired reward', command[0], i_episode)    # write loss to a graph
            writer.add_scalar('Command horizon', command[1], i_episode)    # write loss to a graph
            observation = env.reset()
            episode_mem = {'observation':[],
                            'action':[],
                            'reward':[]}
            done=False
            while not done:
                network_input = torch.tensor(np.append(observation, command)).double()
                action = policy(network_input)
                new_observation, reward, done, info = env.step(action)
                
                episode_mem['observation'].append(observation)
                episode_mem['action'].append(action)
                episode_mem['reward'].append(reward)
                
                observation=new_observation
                #command[0]-= reward
                command[0] = max(1, command[0]-reward)
                command[1] = max(1, command[1]-1)
            episode_mem['return']=sum(episode_mem['reward'])
            episode_mem['episode_len']=len(episode_mem['observation'])
            replay_buffer.append(episode_mem)
            i_episode+=1
            writer.add_scalar('Return', sum(episode_mem['reward']), i_episode)    # write loss to a graph
            print("Episode {} finished after {} timesteps. Return = {}".format(i_episode, len(episode_mem['observation']), sum(episode_mem['reward'])))
        env.close()
    except KeyboardInterrupt:
        env.close()
    replay_buffer = sorted(replay_buffer, key=lambda x:x['return'])[-replay_size:]
    return replay_buffer

def sample_command(replay_buffer, last_few):
    if len(replay_buffer)==0:
        return [1, 1]
    else:
        command_samples = replay_buffer[-last_few:]
        lengths = [mem['episode_len'] for mem in command_samples]
        returns = [mem['return'] for mem in command_samples]
        mean_return, std_return = np.mean(returns), np.std(returns)
        command_horizon = np.mean(lengths)
        desired_reward = np.random.uniform(mean_return, mean_return+std_return)
        return [desired_reward, command_horizon]

In [176]:
def train_net(policy_net, replay_buffer, n_updates=100, batch_size=64):
    all_costs = []
    for i in range(n_updates):
        batch_input = np.zeros((batch_size, np.prod(env.observation_space.shape)+2))
        batch_label = np.zeros((batch_size))
        for b in range(batch_size):
            sample_episode = np.random.randint(0, len(replay_buffer))
            sample_t1 = np.random.randint(0, len(replay_buffer[sample_episode]['observation']))
            sample_t2 = len(replay_buffer[sample_episode]['observation'])
            sample_horizon = sample_t2-sample_t1
            sample_mem = replay_buffer[sample_episode]['observation'][sample_t1]
            sample_desired_reward = sum(replay_buffer[sample_episode]['reward'][sample_t1:sample_t2])
            network_input = np.append(sample_mem, [sample_desired_reward, sample_horizon])
            label = replay_buffer[sample_episode]['action'][sample_t1]
            batch_input[b] = network_input
            batch_label[b] = label
        batch_input = torch.tensor(batch_input).double()
        batch_label = torch.tensor(batch_label).long()
        pred = policy_net(batch_input)
        cost = F.cross_entropy(pred, batch_label)
        all_costs.append(cost.item())
        cost.backward()
        policy_net.optimizer.step()
        policy_net.optimizer.zero_grad()
    return np.mean(all_costs)

In [177]:
def create_greedy_policy(policy_network):
    def policy(obs):
        action_prob = policy_network(obs)
        action = np.argmax(action_prob.detach().numpy())
        return action
    return policy

def create_stochastic_policy(policy_network):
    def policy(obs):
        action_prob = policy_network(obs)
        action_sample = np.random.multinomial(1, action_prob.detach().numpy())
        action = np.argmax(action_sample)
        return action
    return policy

In [178]:
i_episode=0
replay_buffer = []
replay_size = 600
last_few = 50

batch_size = 32
n_warm_up_episodes = 50
n_episodes_per_iter = 50
n_updates_per_iter = 200

lr = 0.001
agent = FCNN_AGENT().double()
agent.create_optimizer(lr)

stochastic_policy = create_stochastic_policy(agent)
greedy_policy = create_greedy_policy(agent)

In [179]:
# SET UP TRAINING VISUALISATION
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter() # we will use this to show our models performance on a graph

In [180]:
#Collect warm up episodes
replay_buffer = collect_experience(random_policy, replay_buffer, replay_size, last_few, n_warm_up_episodes)
train_net(agent, replay_buffer, n_updates=n_updates_per_iter, batch_size=batch_size)

Episode 1 finished after 28 timesteps. Return = 28.0
Episode 2 finished after 14 timesteps. Return = 14.0
Episode 3 finished after 14 timesteps. Return = 14.0
Episode 4 finished after 15 timesteps. Return = 15.0
Episode 5 finished after 16 timesteps. Return = 16.0
Episode 6 finished after 15 timesteps. Return = 15.0
Episode 7 finished after 10 timesteps. Return = 10.0
Episode 8 finished after 10 timesteps. Return = 10.0
Episode 9 finished after 51 timesteps. Return = 51.0
Episode 10 finished after 25 timesteps. Return = 25.0
Episode 11 finished after 25 timesteps. Return = 25.0
Episode 12 finished after 24 timesteps. Return = 24.0
Episode 13 finished after 17 timesteps. Return = 17.0
Episode 14 finished after 13 timesteps. Return = 13.0
Episode 15 finished after 11 timesteps. Return = 11.0
Episode 16 finished after 18 timesteps. Return = 18.0
Episode 17 finished after 16 timesteps. Return = 16.0
Episode 18 finished after 18 timesteps. Return = 18.0
Episode 19 finished after 14 timestep

0.6908077066139745

In [181]:
#agent.load_state_dict(torch.load('checkpoints/lunar_lander_32x32_checkpoint_0.pt'))

In [182]:
n_iters = 10000
for i in range(n_iters):
    replay_buffer = collect_experience(stochastic_policy, replay_buffer, replay_size, last_few, n_episodes_per_iter)
    train_net(agent, replay_buffer, n_updates=n_updates_per_iter, batch_size=batch_size)

Episode 51 finished after 18 timesteps. Return = 18.0
Episode 52 finished after 20 timesteps. Return = 20.0
Episode 53 finished after 17 timesteps. Return = 17.0
Episode 54 finished after 15 timesteps. Return = 15.0
Episode 55 finished after 14 timesteps. Return = 14.0
Episode 56 finished after 12 timesteps. Return = 12.0
Episode 57 finished after 14 timesteps. Return = 14.0
Episode 58 finished after 29 timesteps. Return = 29.0
Episode 59 finished after 11 timesteps. Return = 11.0
Episode 60 finished after 23 timesteps. Return = 23.0
Episode 61 finished after 21 timesteps. Return = 21.0
Episode 62 finished after 13 timesteps. Return = 13.0
Episode 63 finished after 13 timesteps. Return = 13.0
Episode 64 finished after 19 timesteps. Return = 19.0
Episode 65 finished after 15 timesteps. Return = 15.0
Episode 66 finished after 15 timesteps. Return = 15.0
Episode 67 finished after 10 timesteps. Return = 10.0
Episode 68 finished after 20 timesteps. Return = 20.0
Episode 69 finished after 11

Episode 251 finished after 9 timesteps. Return = 9.0
Episode 252 finished after 8 timesteps. Return = 8.0
Episode 253 finished after 13 timesteps. Return = 13.0
Episode 254 finished after 12 timesteps. Return = 12.0
Episode 255 finished after 9 timesteps. Return = 9.0
Episode 256 finished after 13 timesteps. Return = 13.0
Episode 257 finished after 9 timesteps. Return = 9.0
Episode 258 finished after 11 timesteps. Return = 11.0
Episode 259 finished after 9 timesteps. Return = 9.0
Episode 260 finished after 9 timesteps. Return = 9.0
Episode 261 finished after 10 timesteps. Return = 10.0
Episode 262 finished after 10 timesteps. Return = 10.0
Episode 263 finished after 8 timesteps. Return = 8.0
Episode 264 finished after 11 timesteps. Return = 11.0
Episode 265 finished after 12 timesteps. Return = 12.0
Episode 266 finished after 9 timesteps. Return = 9.0
Episode 267 finished after 9 timesteps. Return = 9.0
Episode 268 finished after 14 timesteps. Return = 14.0
Episode 269 finished after 1

Episode 449 finished after 10 timesteps. Return = 10.0
Episode 450 finished after 8 timesteps. Return = 8.0
Episode 451 finished after 8 timesteps. Return = 8.0
Episode 452 finished after 10 timesteps. Return = 10.0
Episode 453 finished after 9 timesteps. Return = 9.0
Episode 454 finished after 11 timesteps. Return = 11.0
Episode 455 finished after 9 timesteps. Return = 9.0
Episode 456 finished after 10 timesteps. Return = 10.0
Episode 457 finished after 8 timesteps. Return = 8.0
Episode 458 finished after 9 timesteps. Return = 9.0
Episode 459 finished after 11 timesteps. Return = 11.0
Episode 460 finished after 9 timesteps. Return = 9.0
Episode 461 finished after 9 timesteps. Return = 9.0
Episode 462 finished after 10 timesteps. Return = 10.0
Episode 463 finished after 13 timesteps. Return = 13.0
Episode 464 finished after 9 timesteps. Return = 9.0
Episode 465 finished after 8 timesteps. Return = 8.0
Episode 466 finished after 10 timesteps. Return = 10.0
Episode 467 finished after 12 

Episode 601 finished after 16 timesteps. Return = 16.0
Episode 602 finished after 8 timesteps. Return = 8.0
Episode 603 finished after 15 timesteps. Return = 15.0
Episode 604 finished after 15 timesteps. Return = 15.0
Episode 605 finished after 10 timesteps. Return = 10.0
Episode 606 finished after 9 timesteps. Return = 9.0
Episode 607 finished after 18 timesteps. Return = 18.0
Episode 608 finished after 24 timesteps. Return = 24.0
Episode 609 finished after 10 timesteps. Return = 10.0
Episode 610 finished after 10 timesteps. Return = 10.0
Episode 611 finished after 12 timesteps. Return = 12.0
Episode 612 finished after 8 timesteps. Return = 8.0
Episode 613 finished after 15 timesteps. Return = 15.0
Episode 614 finished after 10 timesteps. Return = 10.0
Episode 615 finished after 9 timesteps. Return = 9.0
Episode 616 finished after 10 timesteps. Return = 10.0
Episode 617 finished after 18 timesteps. Return = 18.0
Episode 618 finished after 16 timesteps. Return = 16.0
Episode 619 finish

Episode 801 finished after 15 timesteps. Return = 15.0
Episode 802 finished after 9 timesteps. Return = 9.0
Episode 803 finished after 10 timesteps. Return = 10.0
Episode 804 finished after 17 timesteps. Return = 17.0
Episode 805 finished after 10 timesteps. Return = 10.0
Episode 806 finished after 30 timesteps. Return = 30.0
Episode 807 finished after 12 timesteps. Return = 12.0
Episode 808 finished after 14 timesteps. Return = 14.0
Episode 809 finished after 16 timesteps. Return = 16.0
Episode 810 finished after 32 timesteps. Return = 32.0
Episode 811 finished after 10 timesteps. Return = 10.0
Episode 812 finished after 22 timesteps. Return = 22.0
Episode 813 finished after 22 timesteps. Return = 22.0
Episode 814 finished after 11 timesteps. Return = 11.0
Episode 815 finished after 11 timesteps. Return = 11.0
Episode 816 finished after 22 timesteps. Return = 22.0
Episode 817 finished after 12 timesteps. Return = 12.0
Episode 818 finished after 10 timesteps. Return = 10.0
Episode 819 

...

Episode 12589 finished after 142 timesteps. Return = 142.0
Episode 12590 finished after 60 timesteps. Return = 60.0
Episode 12591 finished after 115 timesteps. Return = 115.0
Episode 12592 finished after 107 timesteps. Return = 107.0
Episode 12593 finished after 91 timesteps. Return = 91.0
Episode 12594 finished after 98 timesteps. Return = 98.0
Episode 12595 finished after 59 timesteps. Return = 59.0
Episode 12596 finished after 99 timesteps. Return = 99.0
Episode 12597 finished after 89 timesteps. Return = 89.0
Episode 12598 finished after 106 timesteps. Return = 106.0
Episode 12599 finished after 77 timesteps. Return = 77.0
Episode 12600 finished after 57 timesteps. Return = 57.0
Episode 12601 finished after 80 timesteps. Return = 80.0
Episode 12602 finished after 139 timesteps. Return = 139.0
Episode 12603 finished after 152 timesteps. Return = 152.0
Episode 12604 finished after 135 timesteps. Return = 135.0
Episode 12605 finished after 94 timesteps. Return = 94.0
Episode 12606 fin

Episode 12731 finished after 116 timesteps. Return = 116.0
Episode 12732 finished after 138 timesteps. Return = 138.0
Episode 12733 finished after 52 timesteps. Return = 52.0
Episode 12734 finished after 112 timesteps. Return = 112.0
Episode 12735 finished after 106 timesteps. Return = 106.0
Episode 12736 finished after 135 timesteps. Return = 135.0
Episode 12737 finished after 81 timesteps. Return = 81.0
Episode 12738 finished after 139 timesteps. Return = 139.0
Episode 12739 finished after 68 timesteps. Return = 68.0
Episode 12740 finished after 134 timesteps. Return = 134.0
Episode 12741 finished after 166 timesteps. Return = 166.0
Episode 12742 finished after 131 timesteps. Return = 131.0
Episode 12743 finished after 121 timesteps. Return = 121.0
Episode 12744 finished after 125 timesteps. Return = 125.0
Episode 12745 finished after 142 timesteps. Return = 142.0
Episode 12746 finished after 134 timesteps. Return = 134.0
Episode 12747 finished after 133 timesteps. Return = 133.0
Epi

Episode 12876 finished after 113 timesteps. Return = 113.0
Episode 12877 finished after 75 timesteps. Return = 75.0
Episode 12878 finished after 170 timesteps. Return = 170.0
Episode 12879 finished after 107 timesteps. Return = 107.0
Episode 12880 finished after 54 timesteps. Return = 54.0
Episode 12881 finished after 169 timesteps. Return = 169.0
Episode 12882 finished after 88 timesteps. Return = 88.0
Episode 12883 finished after 118 timesteps. Return = 118.0
Episode 12884 finished after 102 timesteps. Return = 102.0
Episode 12885 finished after 118 timesteps. Return = 118.0
Episode 12886 finished after 82 timesteps. Return = 82.0
Episode 12887 finished after 139 timesteps. Return = 139.0
Episode 12888 finished after 185 timesteps. Return = 185.0
Episode 12889 finished after 169 timesteps. Return = 169.0
Episode 12890 finished after 111 timesteps. Return = 111.0
Episode 12891 finished after 110 timesteps. Return = 110.0
Episode 12892 finished after 191 timesteps. Return = 191.0
Episo

Episode 13016 finished after 105 timesteps. Return = 105.0
Episode 13017 finished after 107 timesteps. Return = 107.0
Episode 13018 finished after 90 timesteps. Return = 90.0
Episode 13019 finished after 124 timesteps. Return = 124.0
Episode 13020 finished after 162 timesteps. Return = 162.0
Episode 13021 finished after 135 timesteps. Return = 135.0
Episode 13022 finished after 114 timesteps. Return = 114.0
Episode 13023 finished after 144 timesteps. Return = 144.0
Episode 13024 finished after 90 timesteps. Return = 90.0
Episode 13025 finished after 114 timesteps. Return = 114.0
Episode 13026 finished after 131 timesteps. Return = 131.0
Episode 13027 finished after 97 timesteps. Return = 97.0
Episode 13028 finished after 140 timesteps. Return = 140.0
Episode 13029 finished after 171 timesteps. Return = 171.0
Episode 13030 finished after 122 timesteps. Return = 122.0
Episode 13031 finished after 135 timesteps. Return = 135.0
Episode 13032 finished after 96 timesteps. Return = 96.0
Episo

KeyboardInterrupt: 

In [72]:
visualise_agent(greedy_policy, command=[250, 200], n=5)

Episode 0 finished after 84 timesteps. Return = -134.1428887684655
Episode 1 finished after 62 timesteps. Return = -143.282848576283
Episode 2 finished after 72 timesteps. Return = -0.15610989654310004
Episode 3 finished after 106 timesteps. Return = -72.03684838919166
Episode 4 finished after 105 timesteps. Return = -37.44217835732849


In [73]:
visualise_agent(stochastic_policy, command=[250, 200], n=5)

Episode 0 finished after 97 timesteps. Return = -89.47575725612703


In [220]:
#torch.save(agent.state_dict(), 'checkpoints/lunar_lander_32x32_checkpoint_0.pt')

In [158]:
print([mem['return'] for mem in replay_buffer])

[79.0, 79.0, 79.0, 79.0, 79.0, 79.0, 79.0, 79.0, 79.0, 79.0, 79.0, 79.0, 79.0, 79.0, 79.0, 79.0, 79.0, 79.0, 79.0, 79.0, 79.0, 79.0, 79.0, 79.0, 79.0, 79.0, 79.0, 79.0, 79.0, 79.0, 79.0, 79.0, 79.0, 79.0, 79.0, 79.0, 80.0, 80.0, 80.0, 80.0, 80.0, 80.0, 80.0, 80.0, 80.0, 80.0, 80.0, 80.0, 80.0, 80.0, 80.0, 80.0, 80.0, 80.0, 80.0, 80.0, 80.0, 80.0, 80.0, 80.0, 80.0, 80.0, 80.0, 80.0, 80.0, 80.0, 80.0, 80.0, 80.0, 80.0, 80.0, 80.0, 80.0, 80.0, 80.0, 80.0, 80.0, 80.0, 80.0, 80.0, 80.0, 80.0, 80.0, 80.0, 80.0, 80.0, 80.0, 80.0, 80.0, 80.0, 81.0, 81.0, 81.0, 81.0, 81.0, 81.0, 81.0, 81.0, 81.0, 81.0, 81.0, 81.0, 81.0, 81.0, 81.0, 81.0, 81.0, 81.0, 81.0, 81.0, 81.0, 81.0, 81.0, 81.0, 81.0, 81.0, 81.0, 81.0, 81.0, 81.0, 81.0, 81.0, 81.0, 81.0, 81.0, 81.0, 81.0, 81.0, 81.0, 81.0, 81.0, 81.0, 81.0, 81.0, 81.0, 81.0, 81.0, 81.0, 81.0, 81.0, 81.0, 81.0, 81.0, 81.0, 81.0, 81.0, 81.0, 82.0, 82.0, 82.0, 82.0, 82.0, 82.0, 82.0, 82.0, 82.0, 82.0, 82.0, 82.0, 82.0, 82.0, 82.0, 82.0, 82.0, 82.0, 82.0, 82.

In [ ]:
def train_net(policy_net, replay_buffer, n_updates=100, batch_size=64):
    all_costs = []
    for i in range(n_updates):
        batch_input = np.zeros((batch_size, np.prod(env.observation_space.shape)+2))
        batch_label = np.zeros((batch_size))
        for b in range(batch_size):
            sample_episode = np.random.randint(0, len(replay_buffer))
            sample_horizon = np.random.randint(1, len(replay_buffer[sample_episode]['observation'])+1)
            sample_mem_idx = np.random.randint(0, len(replay_buffer[sample_episode]['observation'])+1-sample_horizon)
            sample_mem = replay_buffer[sample_episode]['observation'][sample_mem_idx]
            sample_desired_reward = sum(replay_buffer[sample_episode]['reward'][sample_mem_idx:sample_mem_idx+sample_horizon])
            network_input = np.append(sample_mem, [sample_desired_reward, sample_horizon])
            label = replay_buffer[sample_episode]['action'][sample_mem_idx]
            batch_input[b] = network_input
            batch_label[b] = label
        batch_input = torch.tensor(batch_input).double()
        batch_label = torch.tensor(batch_label).long()
        pred = policy_net(batch_input)
        cost = F.cross_entropy(pred, batch_label)
        all_costs.append(cost.item())
        cost.backward()
        policy_net.optimizer.step()
        policy_net.optimizer.zero_grad()
    return np.mean(all_costs)

In [8]:
def train_net(policy_net, episode_mem, n_samples = 5): #stochastic gradient descent
    all_costs = []
    for i in range(n_samples):
        sample_horizon = np.random.randint(1, len(episode_mem['observation'])+1)
        sample_mem_idx = np.random.randint(0, len(episode_mem['observation'])+1-sample_horizon)
        sample_mem = episode_mem['observation'][sample_mem_idx]
        sample_desired_reward = sum(episode_mem['reward'][sample_mem_idx:sample_mem_idx+sample_horizon])
        network_input = torch.tensor(np.append(sample_mem, [sample_desired_reward, sample_horizon])).double()
        label = torch.tensor([episode_mem['action'][sample_mem_idx]]).double()
        
        pred = policy_net(network_input)
        cost = F.binary_cross_entropy(pred, label)
        all_costs.append(cost.item())
        cost.backward()
        policy_net.optimizer.step()
        policy_net.optimizer.zero_grad()
    return np.mean(all_costs)
    

In [33]:
def train(policy_net, n_episodes=100):
    global i_episode
    global epsilon
    try:
        for _ in range(n_episodes):
            observation = env.reset()
            episode_mem = {'observation':[],
                            'action':[],
                            'reward':[],
                            'done':[]}
            done=False
            while not done:
                network_input = torch.tensor(np.append(observation, [desired_reward, command_horizon])).double()
                action_prob = policy_net(network_input)
                action = np.random.binomial(1, action_prob.item())
                #action = int(action_prob.item()>0.5)
                if np.random.rand()<epsilon: action = np.random.randint(0, 2)
                new_observation, reward, done, info = env.step(action)
                
                episode_mem['observation'].append(observation)
                episode_mem['action'].append(action)
                episode_mem['reward'].append(reward)
                episode_mem['done'].append(done)
                
                observation=new_observation
                epsilon*=0.999
            episode_mem['return']=sum(episode_mem['reward'])
            episode_mem['episode_len']=len(episode_mem['observation'])
            mean_cost = train_net(policy_net, episode_mem)
            
            i_episode+=1
            print("Episode {} finished after {} timesteps. Epsilon={} Mean Cost={}".format(i_episode, len(episode_mem['observation']), epsilon, mean_cost))
        env.close()
    except KeyboardInterrupt:
        env.close()